In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

df=pd.read_csv('train.csv')
df

In [ ]:
binary_change= {
    'satisfied': 1,
    'neutral or dissatisfied': 0,
}

df['satisfaction_binary'] = df['satisfaction'].replace(binary_change)
df.drop(columns=['Unnamed: 0'], inplace=True)

# Assuming 'df' is your DataFrame with an 'Age' column

# 1. Define the Conditions
# Each condition must be a Boolean Series (True/False)
conditions = [
    (df['Age'] < 18),
    (df['Age'] >= 18) & (df['Age'] < 30),
    (df['Age'] >= 30) & (df['Age'] < 40),
    (df['Age'] >= 40) & (df['Age'] < 50),
    (df['Age'] >= 50) & (df['Age'] < 60),
    (df['Age'] >= 60) & (df['Age'] < 70),
    (df['Age'] >= 70)
]

# 2. Define the Corresponding Outputs (Labels)
choices = [
    'Minor',
    '18 to 30',
    '30 to 40',
    '40 to 50',
    '50 to 60',
    '60 to 70',
    '70 +'
]

# 3. Apply np.select()
# 'Other' is the default value if none of the conditions are True (e.g., if Age is NaN)
df['age groups'] = np.select(conditions, choices, default='Unknown')

print("Age Group creation successful.")

# Assuming 'df' is your DataFrame with an 'Age' column

# 1. Define the Conditions
# Each condition must be a Boolean Series (True/False)
conditions = [
    (df['Flight Distance'] < 750),
    (df['Flight Distance'] >= 750) & (df['Flight Distance'] < 1500),
    (df['Flight Distance'] >= 1500) & (df['Flight Distance'] < 2250),
    (df['Flight Distance'] >= 2250) & (df['Flight Distance'] < 3000),
    (df['Flight Distance'] >= 3000)
]

# 2. Define the Corresponding Outputs (Labels)
choices = [
    'Short, until 750 miles',
    'Medium-short, until 1500 miles',
    'Medium-Long, until 2250 miles',
    'Long, until 3000 miles',
    'Very long, more than 3000 miles'
]

# 3. Apply np.select()
# 'Other' is the default value if none of the conditions are True (e.g., if Age is NaN)
df['Flight Distance_group'] = np.select(conditions, choices, default='Unknown')

print("Flight distance group Group creation successful.")

In [ ]:
dfSatisfied=df[df['satisfaction_binary'] == 1]
dfDisatisfied=df[df['satisfaction_binary'] == 0]

In [ ]:
numeric_cols_df = df.select_dtypes(include=np.number)

# Exclude 'id' and any other non-metric numeric columns (like Age)
metrics_to_analyze = [
    'Flight Distance',
    'Inflight wifi service',
    'Departure/Arrival time convenient',
    'Ease of Online booking',
    'Gate location',
    'Food and drink',
    'Online boarding',
    'Seat comfort',
    'Inflight entertainment',
    'On-board service',
    'Leg room service',
    'Baggage handling',
    'Checkin service',
    'Inflight service',
    'Cleanliness',
    'Departure Delay in Minutes',
    'Arrival Delay in Minutes',
    'Age' # Including age as a numeric metric to analyze
]

# Filter down to the columns that actually exist and are numeric
analysis_cols = [col for col in metrics_to_analyze if col in numeric_cols_df.columns]

# --- 2. Melt the DataFrame ---
df_melted = df.melt(
    id_vars=['satisfaction'],
    value_vars=analysis_cols,
    var_name='Metric',              # This column now holds the name of the original column
    value_name='Value'              # This column now holds the numeric data
)

# --- 3. Create the Pivot Table ---
pivot_stats = pd.pivot_table(
    df_melted,
    index=['Metric'],
    columns=['satisfaction'],       # 'satisfied' and 'unsatisfied' become columns
    values='Value',
    aggfunc=['mean', 'median', 'std'] # The desired statistics
)

# --- 4. Re-orient to Vertical View ---
# Stacking moves the first level of the column index (mean, median, std) to the rows
vertical_stats_table = pivot_stats.stack(level=0)

# Rename the new index level for clarity
vertical_stats_table.index.set_names(['Metric', 'Statistic'], inplace=True)

# Display the final vertical table
vertical_stats_table

In [ ]:
# --- 1. Define the List of Columns to Include (All 1-5 Scale Metrics) ---
filtered_metrics_1to5 = [
    'Inflight wifi service',
    'Departure/Arrival time convenient',
    'Ease of Online booking',
    'Gate location',
    'Food and drink',
    'Online boarding',
    'Seat comfort',
    'Inflight entertainment',
    'On-board service',
    'Leg room service',
    'Baggage handling',
    'Checkin service',
    'Inflight service',
    'Cleanliness',
]

# --- 2. Filter the df_melted DataFrame ---
# Only keep rows where the 'Metric' is in the list above
df_melted_filtered_1to5 = df_melted[df_melted['Metric'].isin(filtered_metrics_1to5)]


# --- 3. Define the CORRECTED Palette Dictionary (as previously fixed) ---
correct_palette = {
    'satisfied': 'tab:blue',
    'neutral or dissatisfied': 'tab:orange'
}

# --- 4. Create the Grouped Bar Plot with the FILTERED data ---
plt.figure(figsize=(16, 8))

sns.barplot(
    data=df_melted_filtered_1to5, # Using the filtered data
    x='Metric',
    y='Value',
    hue='satisfaction',
    errorbar='sd',
    palette=correct_palette
)

# --- 5. Format the Chart ---
plt.title('Mean Scores for In-Flight and Check-in Metrics by Satisfaction Status', fontsize=16)
plt.ylabel('Mean Rating (1-5 Scale)', fontsize=14) # Changed Y-label for clarity
plt.xlabel('Metric', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.ylim(0, 5.5) # Set Y-axis limit appropriate for a 1-5 scale
plt.tight_layout()
plt.show()

In [ ]:
# dfSatisfied_disloyal=df[df['satisfaction_binary'] == 1 & 'Customer Type'=='disloyal Customer']
dfDisatisfied_disloyal=df[(df['satisfaction_binary'] == 0) & (df['Customer Type']=='disloyal Customer')]
dfDisloyal=df[df['Customer Type']=='disloyal Customer']

In [ ]:
legend_colors = ['#1f77b4', '#ff7f0e', '#2ca02c'] 

# Define the corresponding labels
legend_labels = ['Category A', 'Category B', 'Category C']

In [ ]:
metric_columns = [
     'Inflight wifi service',
     'Departure/Arrival time convenient',
     'Ease of Online booking',
     'Gate location',
     'Food and drink',
     'Online boarding',
     'Seat comfort',
     'Inflight entertainment',
     'On-board service',
     'Leg room service',
     'Baggage handling',
     'Checkin service',
     'Inflight service',
     'Cleanliness',
     'Departure Delay in Minutes',
     'Arrival Delay in Minutes',
    # Add all other numeric metrics you want to compare
]
# --- 1. Initialize Lists for Storing Results ---
metric_names = []
p_values = []
is_significant = []
alpha = 0.05 # Significance level

# --- 2. Iterate, Test, and Collect Results ---
for col in metric_columns:
    # Safely convert to numeric and drop NaNs for both comparison groups
    # NOTE: You must have 'dfDisloyal' and 'dfDisatisfied_disloyal' defined outside this loop.
    df_1_data = pd.to_numeric(dfDisloyal[col], errors='coerce').dropna()
    df_2_data = pd.to_numeric(dfDisatisfied_disloyal[col], errors='coerce').dropna()
    
    # Skip if groups are too small or empty after cleaning
    if df_1_data.size < 2 or df_2_data.size < 2:
        print(f"Skipping {col}: Insufficient data.")
        continue

    # Perform Welch's T-Test
    t_statistic, p_value = stats.ttest_ind(
        a=df_1_data,
        b=df_2_data,
        equal_var=False
    )

    # Store results in the lists
    metric_names.append(col)
    p_values.append(p_value)
    is_significant.append(p_value <= alpha)

    # --- Output the Results (Optional, as requested) ---
    print(f"--- T-Test Results for {col} ---")
    print(f"df_1 Mean: {df_1_data.mean():.3f}")
    print(f"df_2 Mean: {df_2_data.mean():.3f}")
    print(f"T-Statistic: {t_statistic:.4f}")
    print(f"P-Value: {p_value:.10f}")
    print("-" * 38)

# --- 3. Create DataFrame for Plotting ---
p_values_df = pd.DataFrame({
    'Metric': metric_names,
    'P_Value': p_values,
    'Is_Significant': is_significant
})

# # Sort the results by P-Value for a cleaner chart
# p_values_df = p_values_df.sort_values(by='P_Value', ascending=True)

# --- 4. Plot the P-Values ---

# Define colors for bars based on significance
# --- Define the mapping dictionary ---
# Create the dictionary that maps the unique hue values (True/False) to colors
palette_map = {
    True: 'darkred',   # For bars where Is_Significant is True
    False: 'skyblue'  # For bars where Is_Significant is False
}

plt.figure(figsize=(14, 8))
ax = sns.barplot(
    data=p_values_df,
    x='Metric',
    y='P_Value',
    hue='Is_Significant',
    palette=palette_map, # <-- FIX: Pass the dictionary here
    dodge=False,
)
# Add horizontal line at p=0.05
plt.axhline(alpha, color='green', linestyle='--', linewidth=2, label=r'$\alpha = 0.05$ Threshold')

# --- Custom Legend ---
from matplotlib.patches import Patch
custom_handles = [Patch(facecolor=c, label=l) for c, l in zip(legend_colors, legend_labels)]
significance_line = plt.Line2D([0], [0], color='green', linestyle='--', linewidth=2)
final_handles = custom_handles + [significance_line]
final_labels = legend_labels + [r'$\alpha = 0.05$ Threshold']

plt.legend(handles=final_handles, labels=final_labels, title='T-Test Result', loc='upper right')
ax.get_legend().remove() 

# --- Formatting ---
plt.title('P-Values from T-Tests for only Disloyal vs Disatisfied disloyal customers', fontsize=16)
plt.ylabel('P-Value', fontsize=14)
plt.xlabel('Metric', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.ylim(0, 1.0)
plt.tight_layout()
plt.show()

In [ ]:
# --- 1. Initialize Lists for Storing Results ---
metric_names = []
p_values = []
is_significant = []
alpha = 0.05 # Significance level

# --- 2. Iterate, Test, and Collect Results ---
for col in metric_columns:
    # Safely convert to numeric and drop NaNs for both comparison groups
    # NOTE: You must have 'dfDisloyal' and 'dfDisatisfied_disloyal' defined outside this loop.
    df_1_data = pd.to_numeric(dfDisatisfied[col], errors='coerce').dropna()
    df_2_data = pd.to_numeric(dfDisatisfied_disloyal[col], errors='coerce').dropna()
    
    # Skip if groups are too small or empty after cleaning
    if df_1_data.size < 2 or df_2_data.size < 2:
        print(f"Skipping {col}: Insufficient data.")
        continue

    # Perform Welch's T-Test
    t_statistic, p_value = stats.ttest_ind(
        a=df_1_data,
        b=df_2_data,
        equal_var=False
    )

    # Store results in the lists
    metric_names.append(col)
    p_values.append(p_value)
    is_significant.append(p_value <= alpha)

    # --- Output the Results (Optional, as requested) ---
    print(f"--- T-Test Results for {col} ---")
    print(f"df_1 Mean: {df_1_data.mean():.3f}")
    print(f"df_2 Mean: {df_2_data.mean():.3f}")
    print(f"T-Statistic: {t_statistic:.4f}")
    print(f"P-Value: {p_value:.10f}")
    print("-" * 38)

# --- 3. Create DataFrame for Plotting ---
p_values_df = pd.DataFrame({
    'Metric': metric_names,
    'P_Value': p_values,
    'Is_Significant': is_significant
})

# # Sort the results by P-Value for a cleaner chart
# p_values_df = p_values_df.sort_values(by='P_Value', ascending=True)

# --- 4. Plot the P-Values ---

# Define colors for bars based on significance
# --- Define the mapping dictionary ---
# Create the dictionary that maps the unique hue values (True/False) to colors
palette_map = {
    True: 'darkred',   # For bars where Is_Significant is True
    False: 'skyblue'  # For bars where Is_Significant is False
}

plt.figure(figsize=(14, 8))
ax = sns.barplot(
    data=p_values_df,
    x='Metric',
    y='P_Value',
    hue='Is_Significant',
    palette=palette_map, # <-- FIX: Pass the dictionary here
    dodge=False,
)
# Add horizontal line at p=0.05
plt.axhline(alpha, color='green', linestyle='--', linewidth=2, label=r'$\alpha = 0.05$ Threshold')

# --- Custom Legend ---
from matplotlib.patches import Patch
custom_handles = [Patch(facecolor=c, label=l) for c, l in zip(legend_colors, legend_labels)]
significance_line = plt.Line2D([0], [0], color='green', linestyle='--', linewidth=2)
final_handles = custom_handles + [significance_line]
final_labels = legend_labels + [r'$\alpha = 0.05$ Threshold']

plt.legend(handles=final_handles, labels=final_labels, title='T-Test Result', loc='upper right')
ax.get_legend().remove() 

# --- Formatting ---
plt.title('P-Values from T-Tests for only Disatisfied vs Disatisfied disloyal customers', fontsize=16)
plt.ylabel('P-Value', fontsize=14)
plt.xlabel('Metric', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.ylim(0, 1.0)
plt.tight_layout()
plt.show()

In [ ]:
customer_type_counts = df['Customer Type'].value_counts()

plt.figure(figsize=(8, 8))
plt.pie(
    customer_type_counts,
    labels=customer_type_counts.index,
    # This is the key part: autopct='%1.1f%%' formats the percentage
    autopct='%1.1f%%',
    startangle=90,
    wedgeprops={'edgecolor': 'black'}
)
plt.title('Loyalty distribution on the entire population')
plt.axis('equal') # Ensures the pie chart is circular
plt.savefig('customer_type_pie_chart.png')

In [ ]:
customer_type_counts = dfDisatisfied['Customer Type'].value_counts()

plt.figure(figsize=(8, 8))
plt.pie(
    customer_type_counts,
    labels=customer_type_counts.index,
    # This is the key part: autopct='%1.1f%%' formats the percentage
    autopct='%1.1f%%',
    startangle=90,
    wedgeprops={'edgecolor': 'black'}
)
plt.title('Loyalty distribution on Disatisfied customers')
plt.axis('equal') # Ensures the pie chart is circular
plt.savefig('customer_type_pie_chart.png')

In [ ]:
customer_type_counts = dfSatisfied['Customer Type'].value_counts()

plt.figure(figsize=(8, 8))
plt.pie(
    customer_type_counts,
    labels=customer_type_counts.index,
    # This is the key part: autopct='%1.1f%%' formats the percentage
    autopct='%1.1f%%',
    startangle=90,
    wedgeprops={'edgecolor': 'black'}
)
plt.title('Loyalty distribution on Satisfied customers')
plt.axis('equal') # Ensures the pie chart is circular
plt.savefig('customer_type_pie_chart.png')

In [ ]:
# --- 1. DEFINE AND APPLY CUSTOM GROUPS (using the logic you provided) ---

# --- Define the Conditions ---
# Assuming you are applying this logic to the dfDisatisfied DataFrame
conditions = [
    (dfDisatisfied['Flight Distance'] < 750),
    (dfDisatisfied['Flight Distance'] >= 750) & (dfDisatisfied['Flight Distance'] < 1500),
    (dfDisatisfied['Flight Distance'] >= 1500) & (dfDisatisfied['Flight Distance'] < 2250),
    (dfDisatisfied['Flight Distance'] >= 2250) & (dfDisatisfied['Flight Distance'] < 3000),
    (dfDisatisfied['Flight Distance'] >= 3000)
]

# --- Define the Corresponding Labels and Order ---
distance_labels = [
    'Short, until 750 miles',
    'Medium-short, until 1500 miles',
    'Medium-Long, until 2250 miles',
    'Long, until 3000 miles',
    'Very long, more than 3000 miles'
]

# Apply np.select() to create the grouping column
dfDisatisfied['Flight Distance_group'] = np.select(
    conditions, 
    distance_labels, 
    default='Unknown'
)

# --- 2. AUTOMATIC DATA AGGREGATION & PERCENTAGE CALCULATION ---

# Get the count of each Flight Distance group
distance_group_counts_series = dfDisatisfied['Flight Distance_group'].value_counts(dropna=False)

# Convert the Series of counts into a DataFrame (df_counts)
df_counts = distance_group_counts_series.reset_index()
df_counts.columns = ['Flight Distance Group', 'Count']

# Calculate Percentage
total_customers = df_counts['Count'].sum()
df_counts['Percentage'] = (df_counts['Count'] / total_customers) * 100


# --- 3. PLOT BAR CHART WITH ANNOTATIONS ---
plt.figure(figsize=(12, 6))

ax = sns.barplot(
    x='Flight Distance Group',
    y='Percentage',
    data=df_counts,
    order=distance_labels, # Ensures the groups plot in logical order
    palette='magma'
)

# Add titles and labels
plt.title('Percentage of Dissatisfied Customers by Flight Distance Group')
plt.xlabel('Flight Distance Group')
plt.ylabel('Percentage (%)')
plt.xticks(rotation=15, ha='right') # Slight rotation for readability

# Add percentage values on top of the bars (Annotations)
for p in ax.patches:
    ax.annotate(f'{p.get_height():.1f}%',
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center',
                xytext=(0, 9),
                textcoords='offset points')

# Adjust y-limit to ensure annotations are visible
plt.ylim(0, df_counts['Percentage'].max() * 1.20)
plt.grid(axis='y', linestyle='--')
plt.tight_layout()

# Save the plot
plt.savefig('dissatisfied_customers_flight_distance_percentage_bar_chart_final.png')
plt.show()

In [ ]:
# --- 1. AUTOMATIC DATA AGGREGATION ---
# Get the count of each Gender from the dfDisatisfied DataFrame
gender_counts_series = dfDisatisfied['Gender'].value_counts()

# Convert the Series of counts into a DataFrame (df_counts)
df_counts = gender_counts_series.reset_index()
df_counts.columns = ['Gender', 'Count']

# Sort the data by count (or percentage) before plotting
df_counts = df_counts.sort_values(by='Count', ascending=False)


# --- 2. CALCULATE PERCENTAGE ---
total_customers = df_counts['Count'].sum()
df_counts['Percentage'] = (df_counts['Count'] / total_customers) * 100


# --- 3. PLOT BAR CHART ---
plt.figure(figsize=(8, 6))

ax = sns.barplot(
    x='Gender',
    y='Percentage',
    data=df_counts,
    # Order is naturally handled by the prior sorting of df_counts
    palette='Set2' # Used a different palette (Set2)
)

# Add titles and labels
plt.title('Percentage of Dissatisfied Customers by Gender')
plt.xlabel('Gender')
plt.ylabel('Percentage (%)')
plt.xticks(rotation=0, ha='center') # No rotation needed

# Add percentage values on top of the bars (Annotations)
for p in ax.patches:
    ax.annotate(f'{p.get_height():.1f}%',
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center',
                xytext=(0, 9),
                textcoords='offset points')

# Adjust y-limit for the annotations
plt.ylim(0, df_counts['Percentage'].max() * 1.15)
plt.grid(axis='y', linestyle='--')
plt.tight_layout()

# Save the plot
plt.savefig('dissatisfied_customers_gender_percentage_bar_chart.png')

In [ ]:
# --- 1. AUTOMATIC DATA AGGREGATION ---
# Get the count of each age group from the dfDisatisfied DataFrame
age_group_counts_series = dfDisatisfied['age groups'].value_counts()

# Convert the Series of counts into a DataFrame (df_counts) for plotting
df_counts = age_group_counts_series.reset_index()
df_counts.columns = ['Age Group', 'Count']


# --- 2. CALCULATE PERCENTAGE ---
total_customers = df_counts['Count'].sum()
df_counts['Percentage'] = (df_counts['Count'] / total_customers) * 100

# Optional: Ensure all groups are present, even if count is 0,
# and ensure they are properly ordered by the chronological order of ages.
# We will use the 'order' parameter in sns.barplot for this.
age_order = ['Minor', '18 to 30', '30 to 40', '40 to 50', '50 to 60', '60 to 70', '70 +']


# --- 3. PLOT BAR CHART ---
plt.figure(figsize=(10, 6))

ax = sns.barplot(
    x='Age Group',
    y='Percentage',
    data=df_counts,
    order=age_order,       # Use the defined chronological order
    palette='viridis'
)

# Add titles and labels
plt.title('Percentage of Dissatisfied Customers by Age Group (Chronological Order)')
plt.xlabel('Age Group')
plt.ylabel('Percentage (%)')
plt.xticks(rotation=45, ha='right')

# Add percentage values on top of the bars (Annotations)
for p in ax.patches:
    ax.annotate(f'{p.get_height():.1f}%',
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center',
                xytext=(0, 9),
                textcoords='offset points')

# Adjust y-limit for the annotations
plt.ylim(0, df_counts['Percentage'].max() * 1.15)
plt.grid(axis='y', linestyle='--')
plt.tight_layout()

# Save the plot
plt.savefig('dissatisfied_customers_age_percentage_bar_chart_actual.png')
plt.show() # Use plt.show() if running in a script/local environment

In [ ]:
def clean_airloops_data(df):
    # 1. Handle Missing Values: Fill 'Arrival Delay in Minutes' with the median
    median_delay = df['Arrival Delay in Minutes'].median()
    df['Arrival Delay in Minutes'].fillna(median_delay, inplace=True)

    # Note: If 'Inflight wifi service' has nulls (which it often does),
    # we can fill those with 0, assuming 'No Service' or 'Not Applicable'.
    if 'Inflight wifi service' in df.columns and df['Inflight wifi service'].isnull().any():
        df['Inflight wifi service'].fillna(0, inplace=True)

    # 2. Drop Unnecessary Columns
    cols_to_drop = ['Unnamed: 0']
    df.drop(columns=cols_to_drop, inplace=True, errors='ignore')

    # 3. Create the Target Variable (satisfaction_binary)
    # Using .map() is a simple, clear way to do binary encoding.
    satisfaction_map = {'satisfied': 1, 'neutral or dissatisfied': 0}
    df['satisfaction_binary'] = df['satisfaction'].map(satisfaction_map)
    df.drop(columns=['satisfaction'], inplace=True)

    # 4. Format Column Names (Lowercase and Underscore)
    df.columns = df.columns.str.lower().str.replace(' ', '_')

    print("\nData cleaning complete. Ready for analysis!")
    return df

# Apply the function to your DataFrame
df_cleaned = clean_airloops_data(df.copy())

In [ ]:
# 1. Check Class Distribution (How many passengers in each class?)
print("--- Class Distribution ---")
print(df_cleaned['class'].value_counts())
print("-" * 30)

# 2. Calculate Mean Satisfaction by Class
# The mean of 'satisfaction_binary' is the percentage of satisfied customers (1s)
satisfaction_by_class = df_cleaned.groupby('class')['satisfaction_binary'].mean().sort_values(ascending=False)

print("\n--- Average Satisfaction by Class ---")
print(satisfaction_by_class)

In [ ]:
# The results you provided, stored in a DataFrame for plotting
satisfaction_data = {
    'Class': ['Business', 'Eco Plus', 'Eco'],
    'Average_Satisfaction': [0.694251, 0.246064, 0.186138]
}
plot_df = pd.DataFrame(satisfaction_data)

# Sort the data for a cleaner visual presentation (highest to lowest)
plot_df = plot_df.sort_values(by='Average_Satisfaction', ascending=False)

# Create the plot
plt.figure(figsize=(8, 5))
bars = plt.bar(plot_df['Class'], plot_df['Average_Satisfaction'], color=['#1f77b4', '#ff7f0e', '#2ca02c']) # Custom colors for visual separation

# Add labels and title
plt.title('Average Passenger Satisfaction by Class', fontsize=14)
plt.xlabel('Travel Class', fontsize=12)
plt.ylabel('Average Satisfaction (Percentage)', fontsize=12)
plt.yticks([0.0, 0.2, 0.4, 0.6, 0.8, 1.0], ['0%', '20%', '40%', '60%', '80%', '100%']) # Format y-axis as percentages

# Add the percentage values on top of the bars for clarity
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 0.01, f'{yval:.1%}', ha='center', va='bottom', fontsize=10)

plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()

# Save the plot
plt.savefig('satisfaction_by_class_bar_chart.png')
print("Plot saved as 'satisfaction_by_class_bar_chart.png'")

In [ ]:
#cleaned DataFrame is named 'df_cleaned'

# 1. Check Customer Type Distribution
print("--- Customer Type Distribution ---")
print(df_cleaned['customer_type'].value_counts())
print("-" * 30)

# 2. Calculate Mean Satisfaction by Customer Type
# The mean of 'satisfaction_binary' is the percentage of satisfied customers (1s)
satisfaction_by_customer_type = df_cleaned.groupby('customer_type')['satisfaction_binary'].mean().sort_values(ascending=False)

print("\n--- Average Satisfaction by Customer Type ---")
print(satisfaction_by_customer_type)

In [ ]:
# The results you provided, stored in a DataFrame for plotting
satisfaction_data = {
    'Customer_Type': ['Loyal Customer', 'Disloyal Customer'],
    'Average_Satisfaction': [0.477291, 0.236658]
}
plot_df_h2 = pd.DataFrame(satisfaction_data)

# Sort the data for presentation
plot_df_h2 = plot_df_h2.sort_values(by='Average_Satisfaction', ascending=False)

# Create the plot
plt.figure(figsize=(8, 5))
bars = plt.bar(plot_df_h2['Customer_Type'], plot_df_h2['Average_Satisfaction'], color=['#3a5e8c', '#a63a50'])

# Add labels and title
plt.title('Average Passenger Satisfaction: Loyal vs. Disloyal', fontsize=14)
plt.xlabel('Customer Type', fontsize=12)
plt.ylabel('Average Satisfaction (Percentage)', fontsize=12)
plt.yticks([0.0, 0.1, 0.2, 0.3, 0.4, 0.5], ['0%', '10%', '20%', '30%', '40%', '50%'])

# Add the percentage values on top of the bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 0.01, f'{yval:.1%}', ha='center', va='bottom', fontsize=10)

plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()

# Save the plot
plt.savefig('satisfaction_by_customer_type_bar_chart.png')
print("Plot saved as 'satisfaction_by_customer_type_bar_chart.png'")

In [ ]:
# Assuming your cleaned DataFrame is named 'df_cleaned'

# 1. Identify Service Columns (The columns with rating scores or delay metrics)
# We include all rating columns and the delay columns.
service_columns = [
    'inflight_wifi_service',
    'departure/arrival_time_convenient',
    'ease_of_online_booking',
    'gate_location',
    'food_and_drink',
    'online_boarding',
    'seat_comfort',
    'inflight_entertainment',
    'on-board_service',
    'leg_room_service',
    'baggage_handling',
    'checkin_service',
    'inflight_service',
    'cleanliness',
    'departure_delay_in_minutes',
    'arrival_delay_in_minutes'
]

# 2. Calculate Correlation with satisfaction_binary and sort the results
# The Pearson correlation coefficient (r) is used here.
correlation_results = df_cleaned[service_columns + ['satisfaction_binary']].corr()['satisfaction_binary'].sort_values(ascending=False)

# Remove the correlation of the target variable with itself (which is always 1)
correlation_results = correlation_results.drop('satisfaction_binary')

print("--- Correlation of Service Features with Satisfaction (r value) ---")
print(correlation_results)

In [ ]:
# The results you provided, stored in a Series for plotting
data = {
    'online_boarding': 0.503557,
    'inflight_entertainment': 0.398059,
    'seat_comfort': 0.349459,
    'on-board_service': 0.322383,
    'leg_room_service': 0.313131,
    'cleanliness': 0.305198,
    'inflight_wifi_service': 0.284245,
    'baggage_handling': 0.247749,
    'inflight_service': 0.244741,
    'checkin_service': 0.236174,
    'food_and_drink': 0.209936,
    'ease_of_online_booking': 0.171705,
    'gate_location': 0.000682,
    'departure_delay_in_minutes': -0.050494,
    'departure/arrival_time_convenient': -0.051601,
    'arrival_delay_in_minutes': -0.057435
}

correlation_results = pd.Series(data)

# Create the plot
plt.figure(figsize=(10, 7))

# Define colors: Green for positive correlation, Red for negative
colors = ['g' if x > 0 else 'r' for x in correlation_results]

# Plot the horizontal bar chart
correlation_results.plot(kind='barh', color=colors)

# Add labels and title
plt.title('Correlation (r) of Service Features with Passenger Satisfaction', fontsize=14)
plt.xlabel('Pearson Correlation Coefficient (r)', fontsize=12)
plt.ylabel('Service Feature', fontsize=12)

# Add a vertical line at r=0 for reference
plt.axvline(0, color='gray', linestyle='--', linewidth=0.8)

plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.tight_layout()

# Save the plot
plt.savefig('service_correlation_bar_chart.png')
print("Plot saved as 'service_correlation_bar_chart.png'")

In [ ]:
def analyze_service_correlations_notebook(df):
    """
    Calcula el coeficiente de correlación de Pearson (r) entre
    las columnas de servicio y la variable objetivo satisfaction_binary.
    """
    # Lista de columnas de servicio que ya usaste en tu análisis H3
    service_columns = [
        'inflight_wifi_service',
        'departure/arrival_time_convenient',
        'ease_of_online_booking',
        'gate_location',
        'food_and_drink',
        'online_boarding',
        'seat_comfort',
        'inflight_entertainment',
        'on-board_service',
        'leg_room_service',
        'baggage_handling',
        'checkin_service',
        'inflight_service',
        'cleanliness',
        'departure_delay_in_minutes',
        'arrival_delay_in_minutes'
    ]

    # Calcular la correlación
    correlation_results = df[service_columns + ['satisfaction_binary']].corr()['satisfaction_binary'].sort_values(ascending=False)

    # Eliminar la correlación de la variable objetivo consigo misma
    correlation_results = correlation_results.drop('satisfaction_binary', errors='ignore')
    
    return correlation_results

In [ ]:
# --- 1. Filtrar Data por Clase ---

# Filtrar para Clase Economy
df_eco = df_cleaned[df_cleaned['class'] == 'Eco'].copy()
print(f"Pasajeros Clase Eco para análisis: {len(df_eco)}")

# Filtrar para Clase Eco Plus
df_ecoplus = df_cleaned[df_cleaned['class'] == 'Eco Plus'].copy()
print(f"Pasajeros Clase Eco Plus para análisis: {len(df_ecoplus)}")

print("-" * 30)

# --- 2. Calcular Correlaciones Segmentadas ---

# Análisis de Correlación para CLASE ECO
corr_eco = analyze_service_correlations_notebook(df_eco)
print("\n--- TOP 5 IMPULSORES para CLASE ECO ---")
print(corr_eco.head(5))

print("-" * 30)

# Análisis de Correlación para CLASE ECO PLUS
corr_ecoplus = analyze_service_correlations_notebook(df_ecoplus)
print("\n--- TOP 5 IMPULSORES para CLASE ECO PLUS ---")
print(corr_ecoplus.head(5))

In [ ]:
# --- Data Compilation ---
general_corr = {
    'online_boarding': 0.504,
    'inflight_entertainment': 0.398,
    'seat_comfort': 0.349,
    'inflight_wifi_service': 0.284, 
    'on-board_service': 0.322
}
eco_corr = {
    'inflight_wifi_service': 0.467,
    'online_boarding': 0.316,
    'ease_of_online_booking': 0.233,
    'inflight_entertainment': 0.182,
    'food_and_drink': 0.141
}
ecoplus_corr = {
    'inflight_wifi_service': 0.495,
    'online_boarding': 0.348,
    'inflight_entertainment': 0.328,
    'cleanliness': 0.256,
    'food_and_drink': 0.255
}

combined_df = pd.DataFrame({
    'General': general_corr,
    'Eco': eco_corr,
    'Eco Plus': ecoplus_corr
}).fillna(0).sort_values(by='Eco Plus', ascending=True)

# --- Plotting Setup ---
fig, ax = plt.subplots(figsize=(12, 8))

bar_height = 0.25
y_pos = np.arange(len(combined_df.index))

# Plot bars for the three segments
ax.barh(y_pos + bar_height, combined_df['General'], bar_height, label='General (All Classes)', color='#007AA3', alpha=0.7)
ax.barh(y_pos, combined_df['Eco'], bar_height, label='Eco', color='#FF7F0E', alpha=0.8)
ax.barh(y_pos - bar_height, combined_df['Eco Plus'], bar_height, label='Eco Plus', color='#2CA02C')

# Labels and titles
ax.set_yticks(y_pos)
ax.set_yticklabels(combined_df.index.str.replace('_', ' ').str.title())
ax.set_xlabel('Pearson Correlation Coefficient (r)', fontsize=12)
ax.set_title('Segmented Investment Strategy: Correlation of Drivers by Class', fontsize=16, fontweight='bold')
ax.legend(loc='lower right')
ax.grid(axis='x', linestyle='--', alpha=0.5)

plt.tight_layout()

# --- FIX: Create the directory if it doesn't exist ---
output_dir = 'images'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Now save the file
plt.savefig(os.path.join(output_dir, 'segmented_investment_comparison.png'))
plt.show()

print("Segmented Investment Drivers Comparison chart generated and successfully saved to 'images/segmented_investment_comparison.png'.")

In [ ]:
# --- 1. Data Compilation ---
# Sample Sizes (Weights)
n_eco = 46745
n_ecoplus = 7494
n_total_eco_segment = n_eco + n_ecoplus

# 1. General Correlation (Top 5 from your H3 analysis)
general_corr = {
    'online_boarding': 0.504,
    'inflight_entertainment': 0.398,
    'seat_comfort': 0.349,
    'inflight_wifi_service': 0.284, 
    'on-board_service': 0.322
}

# 2. ECO and ECO PLUS RAW CORRELATIONS
eco_corr_raw = {
    'inflight_wifi_service': 0.467, 'online_boarding': 0.316, 
    'ease_of_online_booking': 0.233, 'inflight_entertainment': 0.182, 
    'food_and_drink': 0.141, 'cleanliness': 0.09 # Adding cleanliness for weighted average
}
ecoplus_corr_raw = {
    'inflight_wifi_service': 0.495, 'online_boarding': 0.348, 
    'inflight_entertainment': 0.328, 'cleanliness': 0.256, 
    'food_and_drink': 0.255, 'ease_of_online_booking': 0.18
}

# 3. CALCULATE THE WEIGHTED AVERAGE for the combined 'Economy Segment'
combined_drivers = {}
all_drivers_keys = set(eco_corr_raw.keys()) | set(ecoplus_corr_raw.keys())

for driver in all_drivers_keys:
    # Get correlation and fill NaN with 0 for drivers not in the segment
    r_eco = eco_corr_raw.get(driver, 0)
    r_ecoplus = ecoplus_corr_raw.get(driver, 0)
    
    # Weighted Average Formula: (r_eco * n_eco + r_ecoplus * n_ecoplus) / n_total
    weighted_r = (r_eco * n_eco + r_ecoplus * n_ecoplus) / n_total_eco_segment
    combined_drivers[driver] = weighted_r

# Select the top 5 for the final chart for clarity
combined_drivers_series = pd.Series(combined_drivers).sort_values(ascending=False).head(5).to_dict()

# --- 4. Final DataFrame for Plotting (General vs. Combined Economy) ---
final_combined_df = pd.DataFrame({
    'General (All Classes)': general_corr,
    'Economy Segment (Eco + Eco Plus)': combined_drivers_series
}).fillna(0).sort_values(by='Economy Segment (Eco + Eco Plus)', ascending=True)

# --- Plotting Setup ---
fig, ax = plt.subplots(figsize=(10, 7))

bar_height = 0.35
y_pos = np.arange(len(final_combined_df.index))

# Plot bars for the two segments
ax.barh(y_pos + bar_height/2, final_combined_df['General (All Classes)'], bar_height, label='General (All Classes)', color='#007AA3', alpha=0.7)
ax.barh(y_pos - bar_height/2, final_combined_df['Economy Segment (Eco + Eco Plus)'], bar_height, label='Economy Segment (Strategic Focus)', color='#DAA520')

# Labels and titles
ax.set_yticks(y_pos)
ax.set_yticklabels(final_combined_df.index.str.replace('_', ' ').str.title())
ax.set_xlabel('Pearson Correlation Coefficient (r)', fontsize=12)
ax.set_title('Investment Strategy: General vs. Combined Economy Segment Drivers', fontsize=14, fontweight='bold')
ax.legend(loc='lower right')
ax.grid(axis='x', linestyle='--', alpha=0.5)

plt.tight_layout()

# --- SAVING THE FILE ---
output_dir = 'images'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
plt.savefig(os.path.join(output_dir, 'simplified_segmented_investment_comparison.png'))
plt.show()

print("Simplified Segmented Investment Drivers Comparison chart code generated.")
print("\n--- Top 5 Drivers for the Combined Economy Segment ---")
print(pd.Series(combined_drivers_series))